In [15]:
import sys
sys.path.append('../')

import numpy as np
from scipy import ndimage

from skimage.filters import sobel_h
from skimage.filters import sobel_v
from scipy import stats


import os
import matplotlib
import matplotlib.pyplot as plt
import scienceplots
from tensorflow.python.client import device_lib

#plt.rcParams['figure.figsize'] = [10,10]

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16

from tensorflow.nn import depthwise_conv2d
from tensorflow.math import multiply, reduce_sum, reduce_mean,reduce_euclidean_norm, sin, cos, abs
from tensorflow import stack, concat, expand_dims

import tensorflow_probability as tfp

from utils.utils import *


plt.style.use(['science', 'ieee'])
plt.rcParams.update({'figure.dpi': '100'})

#### Experiment #1 : $\beta$ accross all layers (Top-10% filters)

In [16]:
model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [17]:
k = 20 # Top 10% highest l2-magnitude filters

In [18]:
#topKfilters(model, l_num-1, k=k)

In [19]:
#np.array(topKfilters(model, l_num-1, k=100))[[1,2,3]]

In [20]:
#list(np.array(topKfilters(model, l_num-1, k=100))[[tf.reshape(tf.where(prv_beta >= 0.5), (-1))]])

In [21]:
#prv_antisym_filters#

In [22]:
np.array(topKfilters(model, 5-1, k=10))

array([  2, 254, 252,  14,  87, 236, 133,  85, 172,  41,  77, 130, 203,
       214,  15,  79, 211, 222, 127,  17,  88, 207,  59,  34, 132])

In [23]:
np.array(topKfilters(model, 5-1, k=100))

array([  2, 254, 252,  14,  87, 236, 133,  85, 172,  41,  77, 130, 203,
       214,  15,  79, 211, 222, 127,  17,  88, 207,  59,  34, 132, 156,
       143,  99, 157, 170, 184,  86,  11, 200, 129, 210, 155, 159,  97,
        50, 206, 227, 189, 168,  12, 160, 126,  27,  18, 208, 111, 191,
        56, 245,  45,  84, 101, 212, 148,  83, 201,  23, 179, 161, 153,
        35, 177,  22, 243, 187, 219,  29, 164,  94,  32,  66,  33,  76,
       193,  46,  91, 194, 162, 216, 107, 119, 118,  60, 224, 246, 244,
        31, 248, 103, 223,  93,  55, 238, 138,   5,  95, 109, 250, 176,
        25, 116, 124,  26,  80, 197, 169,  43, 251,   0, 115, 141, 104,
       166, 221, 232,  51, 131, 145, 136,  72, 105, 158, 230, 228, 142,
         7,  75, 231, 163,  20, 249, 137,  98, 247,  70,  62,  30, 239,
         1, 125,  38, 217,  52, 182, 196, 154, 122, 128, 183, 146,  40,
       192,   3, 198, 213, 240, 205,  28, 229,  63,  65, 218,  68, 144,
       204, 225, 108, 112, 147,  19, 151,  39,  81,  73, 241, 21

In [31]:
conv_layers = []
for l in model.layers:
    if 'conv2d' in str(type(l)).lower():
        conv_layers.append(l)

a_count= []
s_count= []

full_a_a = []
full_a_s = []
full_s_a = []
full_s_s = []

print(f" ============ LAYER {0} , {l.name}==================")
top10F=topKfilters(model,0, k=k)
print(0, top10F)
filters = get_filter(model, 0)[:,:,:, top10F]
s, a = getSymAntiSymTF(filters)
a_e = reduce_euclidean_norm(a, axis=[0,1])**2
#print(a_mag.shape)

s_e = reduce_euclidean_norm(s, axis=[0,1])**2
beta = tf.reduce_mean(a_e/(s_e+a_e), axis=0)


antisym_filters = tf.reshape(tf.where(beta >= 0.5), [-1])
sym_filters = tf.reshape(tf.where(beta < 0.5), [-1])

a_count.append(len(antisym_filters))
s_count.append(len(sym_filters))


for i, l in enumerate(conv_layers[1:]):
    l_num = i+1
    print(f" ============ LAYER {l_num} , {l.name}==================")
    top10F=topKfilters(model,l_num, k=k)
    print(l_num, top10F)
    filters = get_filter(model, l_num)[:,:,:, top10F]
    s, a = getSymAntiSymTF(filters)
    a_e = reduce_euclidean_norm(a, axis=[0,1])**2
    #print(a_mag.shape)

    s_e = reduce_euclidean_norm(s, axis=[0,1])**2
    beta = tf.reduce_mean(a_e/(s_e+a_e), axis=0)


    antisym_filters = tf.reshape(tf.where(beta >= 0.5), [-1])
    sym_filters = tf.reshape(tf.where(beta < 0.5), [-1])

    a_count.append(len(antisym_filters))
    s_count.append(len(sym_filters))

    #print(antisym_filters)

    '''a_mag = reduce_euclidean_norm(a, axis=[0,1, 2])**2
    a_mag = a_mag[top10F]
    s_mag = reduce_euclidean_norm(s, axis=[0,1, 2])**2
    s_mag = s_mag[top10F]'''

    a_a = 0
    a_s = 0
    s_a = 0
    s_s = 0

    prv_top10KF = topKfilters(model, l_num-1, k=k)
    prv_filters = get_filter(model, l_num-1)[:,:,:, prv_top10KF]
    prv_s, prv_a = getSymAntiSymTF(prv_filters)
    prv_a_mag = reduce_euclidean_norm(prv_a, axis=[0,1])**2
    prv_s_mag = reduce_euclidean_norm(prv_s, axis=[0,1])**2
    prv_avg_amag = reduce_mean(prv_a_mag, axis=0).numpy()
    prv_avg_smag = reduce_mean(prv_s_mag, axis=0).numpy()
    prv_beta = reduce_mean(prv_a_mag/(prv_a_mag+prv_s_mag), axis=0).numpy()
    prv_antisym_filters = list(np.array(topKfilters(model, l_num-1, k=100))[[tf.reshape(tf.where(prv_beta >= 0.5), (-1))]])
    prv_sym_filters = list(np.array(topKfilters(model, l_num-1, k=100))[[tf.reshape(tf.where(prv_beta < 0.5), (-1))]])
    for i, idx in enumerate(top10F) :
        #print("Filter ", idx , " : ", topKchannels(l_num+1,idx),"--->",set(topKchannels(l_num+1,idx)).intersection(top10F))
        #Is this filter symetric or antisymmetric ?
        is_anti = beta[i] >= 0.5
        print(is_anti)
        #print()
        # Out of its top-k connections, how many inputs are comming from sym-filters, 
        #                               how many inputs are comming from antisym-filters?

        top_k_connections = list(set(topKchannels(model,l_num,idx, k=k)).intersection(prv_top10KF))
        print("TKC",top_k_connections)
        #Check : does top_k_connections lie in antisym_filters or in sym_filters (or neither) ?  
        #print (top_k_connections, antisym_filters.shape)
        #print(tf.squeeze(antisym_filters) )

        #print("HERE" , set(tf.squeeze(antisym_filters).numpy()).intersection(set(top_k_connections)))
        #print("HERE" , set(tf.squeeze(sym_filters).numpy()).intersection(set(top_k_connections)))
        if is_anti: 
            a_a += len(list(set(prv_antisym_filters).intersection(set(top_k_connections))))
            print(idx, "Anti -> Anti : ", list(set(antisym_filters.numpy()).intersection(set(top_k_connections))))
            s_a += len(list(set(prv_sym_filters).intersection(set(top_k_connections))))
            print(idx,"Anti -> Sym : ", list(set(sym_filters.numpy()).intersection(set(top_k_connections))))

        else:
            a_s += len(list(set((prv_antisym_filters)).intersection(set(top_k_connections))))
            print(idx, "Sym -> Anti : ", list(set((antisym_filters.numpy())).intersection(set(top_k_connections))))
            s_s += len(list(set((prv_sym_filters)).intersection(set(top_k_connections))))
            print(idx, "Sym ->  Sym : ", list(set((sym_filters.numpy())).intersection(set(top_k_connections))))

    num_connections = a_a+a_s+s_a+s_s
    full_a_a.append(a_a/num_connections)
    full_a_s.append(a_s/num_connections)
    full_s_a.append(s_a/num_connections)
    full_s_s.append(s_s/num_connections)

    print("A -> A ",  a_a)
    print("A -> S ",  a_s)
    print("S -> A ",  s_a)
    print("S -> S ",  s_s)
    print(top_k_connections)


'''top10F=topKfilters(model,-1, k=k)
print(l_num, top10F)
filters = get_filter(model, l_num)[:,:,:, top10F]
s, a = getSymAntiSymTF(filters)
a_e = reduce_euclidean_norm(a, axis=[0,1])**2
#print(a_mag.shape)

s_e = reduce_euclidean_norm(s, axis=[0,1])**2
beta = tf.reduce_mean(a_e/(s_e+a_e), axis=0)


antisym_filters = tf.reshape(tf.where(beta >= 0.5), [-1])
sym_filters = tf.reshape(tf.where(beta < 0.5), [-1])

a_count.append(len(antisym_filters))
s_count.append(len(sym_filters))'''


 ============ LAYER 0 , block5_pool==================
0 [10, 45, 51, 0, 47, 62, 3, 37, 58, 26, 41, 34]
 ============ LAYER 1 , block1_conv2==================
1 [18, 59, 19, 0, 29, 10, 30, 47, 52, 55, 13, 20]


c:\Users\AN46710\Anaconda3\envs\masters\lib\site-packages\ipykernel_launcher.py:73: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
c:\Users\AN46710\Anaconda3\envs\masters\lib\site-packages\ipykernel_launcher.py:74: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


tf.Tensor(False, shape=(), dtype=bool)
TKC []
18 Sym -> Anti :  []
18 Sym ->  Sym :  []
tf.Tensor(True, shape=(), dtype=bool)
TKC [34, 3, 37, 41, 45, 51, 58, 62]
59 Anti -> Anti :  [3]
59 Anti -> Sym :  []
tf.Tensor(True, shape=(), dtype=bool)
TKC [0, 34, 3, 37, 41, 10, 45, 62]
19 Anti -> Anti :  [3]
19 Anti -> Sym :  [0, 10]
tf.Tensor(True, shape=(), dtype=bool)
TKC [0, 3, 37, 10, 47, 51, 58]
0 Anti -> Anti :  [3]
0 Anti -> Sym :  [0, 10]
tf.Tensor(True, shape=(), dtype=bool)
TKC [0, 37, 10, 45, 51, 58, 62]
29 Anti -> Anti :  []
29 Anti -> Sym :  [0, 10]
tf.Tensor(True, shape=(), dtype=bool)
TKC [0, 3, 37, 10, 47, 51]
10 Anti -> Anti :  [3]
10 Anti -> Sym :  [0, 10]
tf.Tensor(True, shape=(), dtype=bool)
TKC [34, 3, 37, 10, 45, 51, 26, 62]
30 Anti -> Anti :  [3]
30 Anti -> Sym :  [10]
tf.Tensor(True, shape=(), dtype=bool)
TKC [0, 3, 37, 10, 47, 51, 58]
47 Anti -> Anti :  [3]
47 Anti -> Sym :  [0, 10]
tf.Tensor(False, shape=(), dtype=bool)
TKC [41]
52 Sym -> Anti :  []
52 Sym ->  Sym : 

'top10F=topKfilters(model,-1, k=k)\nprint(l_num, top10F)\nfilters = get_filter(model, l_num)[:,:,:, top10F]\ns, a = getSymAntiSymTF(filters)\na_e = reduce_euclidean_norm(a, axis=[0,1])**2\n#print(a_mag.shape)\n\ns_e = reduce_euclidean_norm(s, axis=[0,1])**2\nbeta = tf.reduce_mean(a_e/(s_e+a_e), axis=0)\n\n\nantisym_filters = tf.reshape(tf.where(beta >= 0.5), [-1])\nsym_filters = tf.reshape(tf.where(beta < 0.5), [-1])\n\na_count.append(len(antisym_filters))\ns_count.append(len(sym_filters))'

In [32]:
s_count

[0, 3, 1, 6, 28, 15, 11, 39, 29, 22, 45, 84, 102]

In [33]:
full_a_a
f = open("figures/vgg16_as_tree.tex", "w+")

f.write("\\begin{tikzpicture}\n\\begin{scope}[every node/.style={circle,thick,draw, align=center}, scale=0.8, transform shape]\n ")

pos = len(conv_layers)
for i in range(len(conv_layers)):
    pos -= 2
    f.write(f"    \\node (A_{i}) at (-2,{pos}) {{A\\\\[-0.7em]{a_count[i]}}};\n")
    f.write(f"    \\node (S_{i}) at (2,{pos}) {{S\\\\[-0.7em]{s_count[i]}}};\n")
    f.write(f"    \\node[align=left, draw=none] at (-4,{pos}) {{{i+1}}};\n")


f.write("\end{scope}")

f.write("\\begin{scope}[>={Stealth[black]},every node/.style={fill=white,circle},every edge/.style={draw=black, thin} ,scale=0.8, transform shape]\n ")
for i in range(len(conv_layers)-1):

    aa_max = "[red, very thick]" if np.argmax([full_a_a[i], full_a_s[i],full_s_a[i],full_s_s[i]]) == 0 else ""
    as_max = "[red, very thick]" if np.argmax([full_a_a[i], full_a_s[i],full_s_a[i],full_s_s[i]]) == 1 else ""
    sa_max = "[red, very thick]" if np.argmax([full_a_a[i], full_a_s[i],full_s_a[i],full_s_s[i]]) == 2 else ""
    ss_max = "[red, very thick]" if np.argmax([full_a_a[i], full_a_s[i],full_s_a[i],full_s_s[i]]) == 3 else ""

    pos -= 2
    f.write(f"        \\path [->] (A_{i}) edge {as_max}  node[near start]  {{${full_a_s[i]:.2f}$}} (S_{i+1});\n")
    f.write(f"        \\path [->] (S_{i}) edge {sa_max}  node[near start] {{${full_s_a[i]:.2f}$}} (A_{i+1});\n")
    f.write(f"        \\path [->] (A_{i}) edge {aa_max}  node[left] {{${full_a_a[i]:.2f}$}} (A_{i+1});\n")
    f.write(f"        \\path [->] (S_{i}) edge {ss_max}  node[right] {{${full_s_s[i]:.2f}$}} (S_{i+1});\n")

f.write("\end{scope}")


f.write("\end{tikzpicture}")


f.close()


In [27]:
full_a_a

[0.8947368421052632,
 0.6857142857142857,
 0.6510416666666666,
 0.24649859943977592,
 0.2766934557979334,
 0.6247422680412371,
 0.5126625598904859,
 0.4638922888616891,
 0.6615491974877878,
 0.5054246165357277,
 0.13489208633093525,
 0.0]